In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import functions as F
import time

In [ ]:

# Create a SparkSession
spark = SparkSession.builder.appName("Home_Sales_Analysis").getOrCreate()


In [ ]:
# Load the home sales data into a DataFrame
url = "https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv"
application_df = spark.read.csv(url, header=True, inferSchema=True)

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop("EIN", "NAME")

In [ ]:
# Cache the DataFrame
application_df.cache()

In [ ]:
# Part 1: Run the query on the cached data and determine the runtime
start_time_cached = time.time()


In [ ]:
query_cached = """
SELECT ROUND(AVG(view), 2) AS average_view
FROM home_sales
WHERE price >= 350000
"""

In [ ]:
result_cached = spark.sql(query_cached)
result_cached.show()

end_time_cached = time.time()
run_time_cached = round(end_time_cached - start_time_cached, 2)
print("Run time for cached query:", run_time_cached, "seconds")


In [ ]:
# Save the DataFrame to Parquet format
application_df.write.partitionBy("year_built").parquet("formatted_home_sales.parquet")

# Create a temporary table for the Parquet data
parquet_df = spark.read.parquet("formatted_home_sales.parquet")
parquet_df.createOrReplaceTempView("parquet_home_sales")


In [ ]:
# Part 2: Run the query on the Parquet data and determine the runtime
start_time_parquet = time.time()


In [ ]:
query_parquet = """
SELECT ROUND(AVG(view), 2) AS average_view
FROM parquet_home_sales
WHERE price >= 350000
"""
result_parquet = spark.sql(query_parquet)
result_parquet.show()

end_time_parquet = time.time()
run_time_parquet = round(end_time_parquet - start_time_parquet, 2)
print("Run time for Parquet query:", run_time_parquet, "seconds")


In [ ]:
# Uncache the home_sales temporary table
application_df.unpersist()

# Verify that the home_sales temporary table is uncached
is_cached = spark.catalog.isCached("home_sales")
print("Is home_sales cached?", is_cached)